In [1]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import joblib
import xlsxwriter
import csv
from collections import OrderedDict

C:\Users\v6210227\AppData\Roaming\Python\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#Queue
queue_list = pd.read_excel('D:/Code/Code/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

In [3]:
def rename_files(data_dir):
    for filename in os.listdir(data_dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(data_dir, filename)
            df = pd.read_csv(file_path)
            title = df['title'][0].strip()
            date_str = df['1_resolve_time'][1].split()[0]
            date = pd.to_datetime(date_str).strftime('%Y-%m-%d')
            new_file_name = f"{title}_{date}.csv"
            
            # Check if file already exists
            if os.path.exists(os.path.join(data_dir, new_file_name)):
                # If file exists, create a new suffix
                i = 1
                while True:
                    new_file_name = f"{title}_{date}_{i}.csv"
                    if not os.path.exists(os.path.join(data_dir, new_file_name)):
                        break
                    i += 1
                    
            new_file_path = os.path.join(data_dir, new_file_name)
            os.rename(file_path, new_file_path)
            
rename_files('D:/TCS Rawdata R1R2/R1')
rename_files('D:/TCS Rawdata R1R2/R2')

C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1917775696.py:5: DtypeWarning: Columns (14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1917775696.py:5: DtypeWarning: Columns (14,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1917775696.py:5: DtypeWarning: Columns (14,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1917775696.py:5: DtypeWarning: Columns (19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1917775696.py:5: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.

In [4]:
def convert_datetime(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)    
    
def input_data(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
            df['task_id'] = "id= " + df['task_id'].astype(str)
            df['project_id'] = "id= " + df['project_id'].astype(str)
            df['object_id'] = "id= " + df['object_id'].astype(str)
            df['1_resolve_time'] = df['1_resolve_time'].apply(convert_datetime)
            
            df['source_file'] = os.path.basename(file_path)

            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df


In [5]:
R1_TCS_rawdata = input_data('D:/TCS Rawdata R1R2/R1')
R2_TCS_rawdata = input_data('D:/TCS Rawdata R1R2/R2')

R1_TCS_rawdata.drop_duplicates(subset='task_id',keep='last')
R2_TCS_rawdata.drop_duplicates(subset='task_id',keep='last')

queue_list.columns

C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1871703560.py:35: DtypeWarning: Columns (14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1871703560.py:35: DtypeWarning: Columns (14,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1871703560.py:35: DtypeWarning: Columns (14,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\1871703560.py:35: DtypeWarning: Columns (19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd

Index(['BUSINESS', 'COMPOUND', 'CATEGORY', 'MOD QUEUE ID', 'MOD QUEUE NAME',
       'QA QUEUE ID', 'QA QUEUE NAME', 'QUEUE GROUP', 'STATUS', 'Priority',
       'Latency', 'AHT (secs)', 'Policy_type', 'Classify'],
      dtype='object')

In [6]:
R1_TCS_rawdata['title'].value_counts()

title
VN LL Product Initial Review         260134
VN LL Product High VV 02             120313
VN CB Product Initial Review          90560
VN Product Comment                    23441
VN CB Product High VV 02              10408
VN LL Product Report 02                7460
VN CB Product Buffer                   6242
VN Product Comment Report              3908
VN LL Shop Decoration Risk Review      1272
VN CB Product Report 02                 386
VN LL Product Buffer                     54
VN CB Shop Decoration Risk Review        39
Name: count, dtype: int64

In [7]:
R2_TCS_rawdata['title'].value_counts()

title
QA VN LL Product Initial Review         6132
QA VN LL Product Comment Report         2508
QA VN LL Product Comment                2394
QA VN LL Product Report 02              2394
[QA] VN CB Product High VV 02           2394
[QA] VN LL Product High VV 02           2393
[QA]VN CB Product Buffer                1994
[QA] VN CB Product Initial Review 02    1938
QA VN LL Shop Decoration Risk Review    1271
[QA] VN CB Product Report 02             382
QA VN LL Product Buffer                   54
QA VN CB Shop Decoration Risk Review      39
Name: count, dtype: int64

In [8]:
R1_TCS_rawdata = R1_TCS_rawdata[['task_id', 'project_id', 'title', 'object_id','1_verifier','1_resolve_time','1_duration','1_verify_data']]
R2_TCS_rawdata = R2_TCS_rawdata[['task_id', 'project_id', 'title', 'object_id','1_verifier','1_resolve_time','1_duration','1_verify_data']]


In [9]:
R1_TCS_rawdata_temp = pd.merge(R1_TCS_rawdata,queue_list,how='left',left_on=['title'],right_on=['MOD QUEUE NAME'])

In [10]:
R1_TCS_rawdata_temp[R1_TCS_rawdata_temp['title']=="VN Product Comment Report"]

,task_id,project_id,title,object_id,1_verifier,1_resolve_time,1_duration,1_verify_data,BUSINESS,COMPOUND,...,MOD QUEUE NAME,QA QUEUE ID,QA QUEUE NAME,QUEUE GROUP,STATUS,Priority,Latency,AHT (secs),Policy_type,Classify
496868,id= 7256089596029682177,id= 7060766892780159490,VN Product Comment Report,id= 7256089578237346053,vy.dly@trans-cosmos.com.vn,2023-07-16,6.0,"{""real_assign_time"":1689441496.265,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
496869,id= 7256092291641639425,id= 7060766892780159490,VN Product Comment Report,id= 7256092260691935493,thuc.lt@trans-cosmos.com.vn,2023-07-16,7.0,"{""real_assign_time"":1689440663.602,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
496870,id= 7256092313120670209,id= 7060766892780159490,VN Product Comment Report,id= 7256092289947205893,thuc.lt@trans-cosmos.com.vn,2023-07-16,19.0,"{""real_assign_time"":1689440669.983,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
496871,id= 7256094450215174657,id= 7060766892780159490,VN Product Comment Report,id= 7256094430036230406,vy.dly@trans-cosmos.com.vn,2023-07-16,20.0,"{""real_assign_time"":1689441501.448,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
496872,id= 7256095910659097090,id= 7060766892780159490,VN Product Comment Report,id= 7256095869211132166,vy.dly@trans-cosmos.com.vn,2023-07-16,59.0,"{""real_assign_time"":1689441521.39,""real_resolv...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500771,id= 7263871564452758017,id= 7060766892780159490,VN Product Comment Report,id= 7263871562275619077,sang.vtn@trans-cosmos.com.vn,2023-08-05,66.0,"{""real_assign_time"":1691252744.588,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
500772,id= 7263871980116476417,id= 7060766892780159490,VN Product Comment Report,id= 7263871963129594118,sang.vtn@trans-cosmos.com.vn,2023-08-05,17.0,"{""real_assign_time"":1691252810.166,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
500773,id= 7263872024253137410,id= 7060766892780159490,VN Product Comment Report,id= 7263872017195467014,sang.vtn@trans-cosmos.com.vn,2023-08-05,273.0,"{""real_assign_time"":1691252827.032,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated
500774,id= 7263872657744003586,id= 7060766892780159490,VN Product Comment Report,id= 7263872648349122822,tan.nq@trans-cosmos.com.vn,2023-08-05,10.0,"{""real_assign_time"":1691252920.615,""real_resol...",Local to Local (L2L),LL Product,...,VN Product Comment Report,7181739508772111360.0,QA VN LL Product Comment Report,HCM_EC_L2L_Product,Active,Medium,<1 hr,10.0,Multi-choice,QA moderated


In [11]:
R1_TCS_rawdata_temp.head(1)

,task_id,project_id,title,object_id,1_verifier,1_resolve_time,1_duration,1_verify_data,BUSINESS,COMPOUND,...,MOD QUEUE NAME,QA QUEUE ID,QA QUEUE NAME,QUEUE GROUP,STATUS,Priority,Latency,AHT (secs),Policy_type,Classify
0,id= 7256262128171008513,id= 7120258994144281090,VN CB Product Buffer,id= 1729646742956312636,doan.bnn@trans-cosmos.com.vn,2023-07-16,33.0,"{""real_assign_time"":1689483291.29,""real_resolv...",Cross Border,CB Product,...,VN CB Product Buffer,7127543413397831680.0,[QA]VN CB Product Buffer,HCM_EC_CB_Product,Active,High,<24 hrs,40.0,Multi-choice,QA moderated


In [12]:
Merged_TCS_rawdata = pd.merge(R2_TCS_rawdata,R1_TCS_rawdata_temp,how='left',left_on=['title','object_id'],right_on=['QA QUEUE NAME','object_id'])
Merged_TCS_rawdata.columns

Index(['task_id_x', 'project_id_x', 'title_x', 'object_id', '1_verifier_x',
       '1_resolve_time_x', '1_duration_x', '1_verify_data_x', 'task_id_y',
       'project_id_y', 'title_y', '1_verifier_y', '1_resolve_time_y',
       '1_duration_y', '1_verify_data_y', 'BUSINESS', 'COMPOUND', 'CATEGORY',
       'MOD QUEUE ID', 'MOD QUEUE NAME', 'QA QUEUE ID', 'QA QUEUE NAME',
       'QUEUE GROUP', 'STATUS', 'Priority', 'Latency', 'AHT (secs)',
       'Policy_type', 'Classify'],
      dtype='object')

In [13]:
Merged_TCS_rawdata = Merged_TCS_rawdata[['title_x','project_id_x','1_verifier_y','1_resolve_time_y',
                    'object_id','1_verify_data_y','1_duration_y','1_verifier_x',
                    '1_resolve_time_x','task_id_x','1_verify_data_x']].rename(columns={'1_resolve_time_x':'2_resolve_time',
                                                                                       '1_resolve_time_y':'1_resolve_time',
                                                                                       '1_verify_data_x':'qa_verify_data',
                                                                                       '1_verify_data_y':'mod_verify_data',
                                                                                       'task_id_x':'qa_task_id',
                                                                                       'title_x':'QA QUEUE NAME'})
Merged_TCS_rawdata.head(1)

,QA QUEUE NAME,project_id_x,1_verifier_y,1_resolve_time,object_id,mod_verify_data,1_duration_y,1_verifier_x,2_resolve_time,qa_task_id,qa_verify_data
0,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,thoa.ptk@trans-cosmos.com.vn,2023-07-16,id= 7256305670187403014,"{""real_assign_time"":1689491670.955,""real_resol...",23.0,tram.nn@trans-cosmos.com.vn,2023-07-17,id= 7256554153579741702,"{""real_assign_time"":1689554504.079,""real_resol..."


In [14]:
Merged_TCS_rawdata[Merged_TCS_rawdata['1_verifier_y'].isnull()]['QA QUEUE NAME'].value_counts()

QA QUEUE NAME
QA VN LL Product Initial Review         1861
QA VN LL Product Report 02              1546
QA VN LL Product Comment Report          914
[QA] VN CB Product Initial Review 02       8
QA VN LL Product Comment                   4
[QA] VN LL Product High VV 02              4
[QA] VN CB Product High VV 02              1
Name: count, dtype: int64

In [15]:
Merged_TCS_rawdata = Merged_TCS_rawdata.dropna(subset=['1_verifier_y'])

In [16]:
def filter_time( df,  from_date, to_date):
    Merged_TCS_rawdata['2_resolve_time'] = pd.to_datetime(Merged_TCS_rawdata['2_resolve_time'])
    merged_filter = df[(df['2_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['2_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]

    return merged_filter,from_date,to_date

Merge_TCS,from_date,to_date = filter_time(Merged_TCS_rawdata, '2023-07-30', '2023-08-05')
Merge_TCS

,QA QUEUE NAME,project_id_x,1_verifier_y,1_resolve_time,object_id,mod_verify_data,1_duration_y,1_verifier_x,2_resolve_time,qa_task_id,qa_verify_data
18,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,thoa.ptk@trans-cosmos.com.vn,2023-07-29,id= 7252663616061851397,"{""real_assign_time"":1690601334.231,""real_resol...",60.0,tram.nn@trans-cosmos.com.vn,2023-07-30,id= 7261318316583551494,"{""real_assign_time"":1690679607.73,""real_resolv..."
19,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,thoa.ptk@trans-cosmos.com.vn,2023-07-29,id= 7261164729615025926,"{""real_assign_time"":1690625299.063,""real_resol...",13.0,tram.nn@trans-cosmos.com.vn,2023-07-30,id= 7261318309646287365,"{""real_assign_time"":1690679566.907,""real_resol..."
20,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,viet.na@trans-cosmos.com.vn,2023-07-26,id= 7260076548019013381,"{""real_assign_time"":1690370537.967,""real_resol...",110.0,tram.nn@trans-cosmos.com.vn,2023-08-01,id= 7262056681163489797,"{""real_assign_time"":1690863607.674,""real_resol..."
21,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,quy.hlc@trans-cosmos.com.vn,2023-07-31,id= 7260076548019013381,"{""real_assign_time"":1690803328.786,""real_resol...",182.0,tram.nn@trans-cosmos.com.vn,2023-08-01,id= 7262056681163489797,"{""real_assign_time"":1690863607.674,""real_resol..."
22,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,quy.hlc@trans-cosmos.com.vn,2023-08-05,id= 7260076548019013381,"{""real_assign_time"":1691236914.173,""real_resol...",21.0,tram.nn@trans-cosmos.com.vn,2023-08-01,id= 7262056681163489797,"{""real_assign_time"":1690863607.674,""real_resol..."
...,...,...,...,...,...,...,...,...,...,...,...
30894,[QA]VN CB Product Buffer,id= 7127543413397832194,hau.htc@trans-cosmos.com.vn,2023-08-04,id= 1729674750825106292,"{""real_assign_time"":1691162020.939,""real_resol...",46.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263523987634864641,"{""real_assign_time"":1691242544.707,""real_resol..."
30895,[QA]VN CB Product Buffer,id= 7127543413397832194,thinh.vg@trans-cosmos.com.vn,2023-08-04,id= 1729674190962592377,"{""real_assign_time"":1691155336.755,""real_resol...",184.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263523992315691522,"{""real_assign_time"":1691242703.675,""real_resol..."
30896,[QA]VN CB Product Buffer,id= 7127543413397832194,toan.dt2@trans-cosmos.com.vn,2023-08-04,id= 1729674388043106534,"{""real_assign_time"":1691158876.22,""real_resolv...",23.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263523999022350850,"{""real_assign_time"":1691242746.078,""real_resol..."
30897,[QA]VN CB Product Buffer,id= 7127543413397832194,mai.lt1@trans-cosmos.com.vn,2023-08-04,id= 1729674387448826086,"{""real_assign_time"":1691158449.683,""real_resol...",21.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263524007926907394,"{""real_assign_time"":1691242766.133,""real_resol..."


In [17]:
Merge_TCS[['mod_verify_data_temp','qa_verify_data_temp']] = Merge_TCS[['mod_verify_data','qa_verify_data']].replace(to_replace=r'^auditStatus: ',value= "Audit Result:: ",regex=True).astype(str)

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()

        if "normal" in s_lower and "remove" not in s_lower and "selfvisible" not in s_lower:
            return "Audit Result: Approve"
        elif "remove" in s_lower or "selfvisible" in s_lower:
            return "Audit Result: Not Approve"

    return s

Merge_TCS.loc[Merge_TCS['QA QUEUE NAME'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']), ['mod_verify_data_temp','qa_verify_data_temp']] \
= Merge_TCS.loc[Merge_TCS['QA QUEUE NAME'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']), ['mod_verify_data_temp','qa_verify_data_temp']].applymap(check_string)

C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\4126381480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merge_TCS[['mod_verify_data_temp','qa_verify_data_temp']] = Merge_TCS[['mod_verify_data','qa_verify_data']].replace(to_replace=r'^auditStatus: ',value= "Audit Result:: ",regex=True).astype(str)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\4126381480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merge_TCS[['mod_verify_data_temp','qa_verify_data_temp']] = Merge_TCS[['mod_verify_dat

In [18]:
keymap = {"Not Approve":['"status":400','"audit_result":400','disapproval', 'disapproved','not approval','not approval','not approve' ,'not approval','disapprove','disapproved','(truncated)'] ,
    "Approve":[ '"status":200' ,'"audit_result":200', 'approval', 'approved','200' ,'200.0' ,'approve']}

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()

        for approval_status, substrings in keymap.items():
            if any(substring in s_lower for substring in substrings):
                return approval_status.capitalize()

    return s

Merge_TCS['1st_result'] = Merge_TCS['mod_verify_data_temp'].apply(check_string)
Merge_TCS['2nd_result'] = Merge_TCS['qa_verify_data_temp'].apply(check_string)

C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\2838743516.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merge_TCS['1st_result'] = Merge_TCS['mod_verify_data_temp'].apply(check_string)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_21348\2838743516.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Merge_TCS['2nd_result'] = Merge_TCS['qa_verify_data_temp'].apply(check_string)


In [19]:
Merge_TCS['1st_result'] .unique()

array(['Not approve', 'Approve'], dtype=object)

In [20]:
Merge_TCS['2nd_result'] .unique()

array(['Not approve', 'Approve'], dtype=object)

In [21]:
Merge_TCS_test = Merge_TCS[['QA QUEUE NAME', 'qa_task_id', 'qa_verify_data', '2nd_result']]
Merge_TCS_test = Merge_TCS_test.loc[Merge_TCS_test['2nd_result'] == 'Not approve']
Merge_TCS_test = Merge_TCS_test.drop_duplicates(subset=['qa_task_id'])

def get_first_not_approve(df):
    return df.iloc[0]

Merge_TCS_test = Merge_TCS_test.groupby('QA QUEUE NAME').apply(get_first_not_approve)
Merge_TCS_test

,QA QUEUE NAME,qa_task_id,qa_verify_data,2nd_result
QA QUEUE NAME,,,,
QA VN CB Shop Decoration Risk Review,QA VN CB Shop Decoration Risk Review,id= 7261318316583551494,"{""real_assign_time"":1690679607.73,""real_resolv...",Not approve
QA VN LL Product Buffer,QA VN LL Product Buffer,id= 7263579203273048578,"{""real_assign_time"":1691222083.429,""real_resol...",Not approve
QA VN LL Product Comment,QA VN LL Product Comment,id= 7261356887549985282,"{""real_assign_time"":1690667433.956,""real_resol...",Not approve
QA VN LL Product Comment Report,QA VN LL Product Comment Report,id= 7261713587570328065,"{""real_assign_time"":1690772468.162,""real_resol...",Not approve
QA VN LL Product Initial Review,QA VN LL Product Initial Review,id= 7261330861549732353,"{""real_assign_time"":1690666435.53,""real_resolv...",Not approve
QA VN LL Shop Decoration Risk Review,QA VN LL Shop Decoration Risk Review,id= 7261298291122405893,"{""real_assign_time"":1690684663.113,""real_resol...",Not approve
[QA] VN CB Product High VV 02,[QA] VN CB Product High VV 02,id= 7261291381182710274,"{""real_assign_time"":1690688147.423,""real_resol...",Not approve
[QA] VN CB Product Initial Review 02,[QA] VN CB Product Initial Review 02,id= 7261306257754063362,"{""real_assign_time"":1690685328.979,""real_resol...",Not approve
[QA] VN CB Product Report 02,[QA] VN CB Product Report 02,id= 7261289487328756225,"{""real_assign_time"":1690684444.579,""real_resol...",Not approve


In [22]:
Merge_TCS = Merge_TCS.replace(['0013', '0012', '0005', '_'], ' ', regex=True)
Merge_TCS['mod_verify_data'] = Merge_TCS['mod_verify_data'].str.lower()
Merge_TCS['qa_verify_data'] = Merge_TCS['qa_verify_data'].str.lower()

In [23]:
Policy_errors_1 = [
    'prohibited products','unsupported products','restricted products','adult nudity and sexual activities','hateful behaviors','minor safety','absolute terms',
    'malicious comparison','title spam','abnormal price','inconsistent information','other languages','misleading functionality and effect','redirect traffic',
    'missing brand authorization','unfair trading','obscuring product images','target to minors','underage commercial behavior','false promise','false promises','sympathy promotion',
    'misleading size/weight/quantity','misleading patent/endorsement/metrics/awards','misleading origin','misleading return & refund service','misleading guarantees and warranties',
    'irrelevant promotion','give away','reproduced content','soliciting mutual interactions','still frame','possible pirated contents','possible pirated content',
    'non product promotion content','vulgar behavior','disturbing animal body','insulting and abusive content','body impurities','dirty environment','eye-catching sensitive content',
    'bind a sell b - prohibited product','misleading logistics','misleading brand','brand circumvention','copycat products','violence',
    'illegal behavior','hatespeech and harassment','possible ip infringement','spam','invalid company name','inaccurate company name','inaccurate company address','missing identity document',
    'expired documentation','black-and-white copy','unclear documentation','invalid watermark or logo','incomplete documentation','inaccurate representative name','inaccurate id number',
    'inaccurate documentation','inaccurate tax name','inaccurate tax number','inaccurate brand name','inaccurate trademark name','inaccurate trademark number','inaccurate application date',
    'inaccurate expiry date','invalid trademark','inaccurate registrant','unqualified destination market','inaccurate class(es) of goods/service','unqualified owner','inaccurate trademark certificate',
    'expired trademark certificate','unclear trademark certificate','inauthentic trademark certificate','unqualified 1st level distributor','inaccurate 1st level distributor letter',
    'expired 1st level distributor letter','unclear 1st level distributor letter','inauthentic 1st level distributor letter','unqualified 2nd level distributor',
    'inaccurate 2nd level distributor letter','expired 2nd level distributor letter','unclear 2nd level distributor letter','inauthentic 2nd level distributor letter',
    'unqualified third level distributor','inaccurate third level distributor letter','expired third level distributor letter','unclear third level distributor letter',
    'inauthentic third level distributor letter','missing buyer information','missing quantity of purchase item','missing purchase item information','missing purchase date',
    'unable to locate brand item product','purchase date mismatch','seller information mismatch','missing price of purchase item','missing seller information',
    'product quantity insufficient','exceeded validity period','altered document','lack of supportive information','unable to recognise source of purchase','underage applicant',
    'gibberish shop name','abnormal company status','traffic redirection','inaccurate company number','oversea personal seller','ps tampering trace','inauthentic documentation',
    'violent extremism','hate speech','unauthorized brand information','impersonation','suicide and dangerous acts','illegal activities and regulated goods',
    'harassment and bullying','child endangerment information','counterfeit product information','inauthentic certification of incorporation','inaccurate register date','erotic nudity','hateful contexts','partner name or logo relates to ''tiktok''','inauthentic work environment','repeated image','inappropriate watermark or logo',
    'incomplete information','inaccurate and misleading review','slightly vulgar','inaccurate package weight information','inappropriate speech','malicious behaviors','vulgar issues',
    'imagery depicting skin defects','imagery depicting indecent body organs','imagery depicting indecent teeth&oral','imagery depicting animal meat/carcasses',
    'imagery depicting unpleasant animal or insect','imagery depicting impurities','horror content','scene of squalor','offensive content','human excrement','mental illness content',
    'unsuitable for recommendation','surprise-based product','vat number check','slide show','counterfeit word indication','brand never produced','bind a sell b-prohibited product',
    'pirated content- dedup recall','pirated content','possible counterfeit product','click bait','click-bait','non shippable products','physical trademark infringement product','incorrect category high risk','low cost content',  
    'gambling related behaviour','information trademark infringement product','physically damaged id','suspicious brand promotion','auction','illegal activities'
    ]


Policy_errors = Policy_errors_1

cols_to_check = ['mod_verify_data','qa_verify_data']
cols_to_return = ['Mod Reason','QA Reason']

queue_policy_type = queue_list[['QA QUEUE NAME','Policy_type']]
   
Merge_TCS = pd.merge(Merge_TCS,queue_policy_type, how='inner', on='QA QUEUE NAME')

for col_idx, col in enumerate(cols_to_check):
    Merge_TCS[cols_to_return[col_idx]] = ''
    for i, row in Merge_TCS.iterrows():
        cell_value = row[col]
        policy_type = row['Policy_type']
        if isinstance(cell_value, str):
            if 'form info' in cell_value:
                form_info_index = cell_value.index('form info')
                cell_value = cell_value[form_info_index:]
                match_list_1 = [match for match in Policy_errors_1 if match in cell_value]
                if match_list_1:
                    match_list = list(OrderedDict.fromkeys(match_list_1))
                    final_reason = ", ".join(match_list).strip()
                    Merge_TCS.at[i, cols_to_return[col_idx]] = final_reason
            elif 'reject reasons' in cell_value:
                form_info_index = cell_value.index('reject reasons')
                cell_value_new = cell_value[:form_info_index]
                match_list_1 = [match for match in Policy_errors_1 if match in cell_value_new]
                if match_list_1:
                    match_list = list(OrderedDict.fromkeys(match_list_1))
                    final_reason = ", ".join(match_list).strip()
                    Merge_TCS.at[i, cols_to_return[col_idx]] = final_reason
            elif 'reject label' in cell_value:
                form_info_index = cell_value.index('reject label')
                cell_value = cell_value[form_info_index:]
                match_list_1 = [match for match in Policy_errors_1 if match in cell_value]
                if match_list_1:
                    match_list = list(OrderedDict.fromkeys(match_list_1))
                    final_reason = ", ".join(match_list).strip()
                    Merge_TCS.at[i, cols_to_return[col_idx]] = final_reason
            else:
                if policy_type == 'Multi-choice':
                    match_list_1 = [match for match in Policy_errors_1 if match in cell_value]
                    if match_list_1:
                        match_list = list(OrderedDict.fromkeys(match_list_1))
                        final_reason = ", ".join(match_list).strip()
                        Merge_TCS.at[i, cols_to_return[col_idx]] = final_reason
                elif policy_type == 'Single-choice':
                    match_list_1 = []
                    for match in Policy_errors_1:
                        if match in cell_value:
                            match_list_1.append(match)
                            break
                    if match_list_1:
                        final_reason = match_list_1[0]
                        Merge_TCS.at[i, cols_to_return[col_idx]] = final_reason
        else:
            continue


Merge_TCS['QA Reason'] = Merge_TCS['QA Reason'].replace("0005","",regex=True).replace("_"," ",regex=True).replace("counterfeit abnorma price product","abnormal price")
Merge_TCS['Mod Reason'] = Merge_TCS['Mod Reason'].replace("0005","",regex=True).replace("_"," ",regex=True).replace("counterfeit abnorma price product","abnormal price")

Merge_TCS['QA Reason'] = Merge_TCS['QA Reason'].apply(lambda x: ", ".join(set(x.split(", "))))
Merge_TCS['Mod Reason'] = Merge_TCS['Mod Reason'].apply(lambda x: ", ".join(set(x.split(", "))))

In [24]:
#Add diff case
Merge_TCS.loc[Merge_TCS['1st_result'] != Merge_TCS['2nd_result'],'Diff case']="FALSE"
Merge_TCS.loc[Merge_TCS['Diff case'].isnull(),'Diff case']="TRUE"

In [25]:
Merge_TCS.columns

Index(['QA QUEUE NAME', 'project_id_x', '1_verifier_y', '1_resolve_time',
       'object_id', 'mod_verify_data', '1_duration_y', '1_verifier_x',
       '2_resolve_time', 'qa_task_id', 'qa_verify_data',
       'mod_verify_data_temp', 'qa_verify_data_temp', '1st_result',
       '2nd_result', 'Policy_type', 'Mod Reason', 'QA Reason', 'Diff case'],
      dtype='object')

In [26]:
Merge_TCS = Merge_TCS[['QA QUEUE NAME','project_id_x','1_verifier_y','1_resolve_time',
                    'object_id','1st_result','Mod Reason','1_duration_y','1_verifier_x',
                    '2_resolve_time','qa_task_id','2nd_result','QA Reason','Diff case']]

In [27]:
Merge_TCS

,QA QUEUE NAME,project_id_x,1_verifier_y,1_resolve_time,object_id,1st_result,Mod Reason,1_duration_y,1_verifier_x,2_resolve_time,qa_task_id,2nd_result,QA Reason,Diff case
0,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,thoa.ptk@trans-cosmos.com.vn,2023-07-29,id= 7252663616061851397,Not approve,incomplete information,60.0,tram.nn@trans-cosmos.com.vn,2023-07-30,id= 7261318316583551494,Not approve,incomplete information,TRUE
1,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,thoa.ptk@trans-cosmos.com.vn,2023-07-29,id= 7261164729615025926,Approve,,13.0,tram.nn@trans-cosmos.com.vn,2023-07-30,id= 7261318309646287365,Approve,,TRUE
2,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,viet.na@trans-cosmos.com.vn,2023-07-26,id= 7260076548019013381,Not approve,incomplete information,110.0,tram.nn@trans-cosmos.com.vn,2023-08-01,id= 7262056681163489797,Not approve,incomplete information,TRUE
3,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,quy.hlc@trans-cosmos.com.vn,2023-07-31,id= 7260076548019013381,Not approve,incomplete information,182.0,tram.nn@trans-cosmos.com.vn,2023-08-01,id= 7262056681163489797,Not approve,incomplete information,TRUE
4,QA VN CB Shop Decoration Risk Review,id= 7182112064087933442,quy.hlc@trans-cosmos.com.vn,2023-08-05,id= 7260076548019013381,Approve,,21.0,tram.nn@trans-cosmos.com.vn,2023-08-01,id= 7262056681163489797,Not approve,incomplete information,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6471,[QA]VN CB Product Buffer,id= 7127543413397832194,hau.htc@trans-cosmos.com.vn,2023-08-04,id= 1729674750825106292,Approve,,46.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263523987634864641,Approve,,TRUE
6472,[QA]VN CB Product Buffer,id= 7127543413397832194,thinh.vg@trans-cosmos.com.vn,2023-08-04,id= 1729674190962592377,Approve,,184.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263523992315691522,Approve,,TRUE
6473,[QA]VN CB Product Buffer,id= 7127543413397832194,toan.dt2@trans-cosmos.com.vn,2023-08-04,id= 1729674388043106534,Approve,,23.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263523999022350850,Approve,,TRUE
6474,[QA]VN CB Product Buffer,id= 7127543413397832194,mai.lt1@trans-cosmos.com.vn,2023-08-04,id= 1729674387448826086,Approve,,21.0,dieu.mtb@trans-cosmos.com.vn,2023-08-05,id= 7263524007926907394,Approve,,TRUE


In [28]:
Merge_TCS.groupby(by=['QA QUEUE NAME'],as_index=False).agg({'object_id':'count'})

,QA QUEUE NAME,object_id
0,QA VN CB Shop Decoration Risk Review,30
1,QA VN LL Product Buffer,42
2,QA VN LL Product Comment,815
3,QA VN LL Product Comment Report,677
4,QA VN LL Product Initial Review,998
5,QA VN LL Product Report 02,15
6,QA VN LL Shop Decoration Risk Review,452
7,[QA] VN CB Product High VV 02,1031
8,[QA] VN CB Product Initial Review 02,567
9,[QA] VN CB Product Report 02,134


In [29]:
Merge_TCS.to_excel(f'D:/Export temp/Product {from_date}_{to_date}.xlsx')